In [1]:
import pandas as pd
import json
import nltk
import numpy as np

In [2]:
dfTrain = pd.read_csv('train.tsv',sep = '\t')
dfTest = pd.read_csv('test.tsv', sep = '\t')

df = dfTrain.append(dfTest,ignore_index=True)

df['setType'] = 'training'
df['setType'][np.isnan(df.label)] = 'test'

<ipython-input-2-a444fbc70dcd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['setType'][np.isnan(df.label)] = 'test'


In [3]:
df

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,setType
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,24,0,5424,170,8,0.152941,0.079130,0.0,training
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,40,0,4973,187,9,0.181818,0.125448,1.0,training
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,55,0,2240,258,11,0.166667,0.057613,1.0,training
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,0,24,0,2737,120,5,0.041667,0.100858,1.0,training
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,14,0,12032,162,10,0.098765,0.082569,0.0,training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10561,http://busy-mommy.com/2012/02/peep-brownie-smo...,7264,"{""title"":""Peep Brownie S mores Busy Mommy An I...",?,?,1.666667,0.376623,0.129870,0.116883,0.090909,...,0,16,0,2772,77,3,0.012987,0.063401,NaN,test
10562,http://www.cannabissearch.com/edibles/cheesecake/,9714,"{""url"":""cannabissearch edibles cheesecake"",""ti...",?,?,1.305556,0.654321,0.123457,0.024691,0.000000,...,0,6,0,6058,81,2,0.333333,0.061995,NaN,test
10563,http://www.tastespotting.com/popular/views/all...,5903,"{""title"":""Most Viewed Submissions All Time mos...",?,?,0.717277,0.291667,0.182292,0.000000,0.000000,...,0,19,0,2876,192,4,0.177083,0.117647,NaN,test
10564,http://lifehacker.com/5839197/how-to-get-a-ful...,3176,"{""title"":""How to Get a Complete Workout with N...",sports,0.424304,0.940000,0.183333,0.066667,0.016667,0.016667,...,1,3,0,21029,180,12,0.333333,0.111966,NaN,test


### Extract features

In [4]:
def getBoilerPlateText(text, obj = 'title'):    
    try:
        result = json.loads(text)[obj].encode('utf-8').strip()    
    except:  # if there is no title, put in blank
        result = ''
    return result

In [5]:
boilerPlateTitles = df.boilerplate.apply(getBoilerPlateText,obj = 'title')
boilerPlateBodies = df.boilerplate.apply(getBoilerPlateText,obj =  'body')

In [15]:
boilerPlateTitles.head(20)

0     b'IBM Sees Holographic Calls Air Breathing Bat...
1     b'The Fully Electronic Futuristic Starting Gun...
2     b"Fruits that Fight the Flu fruits that fight ...
3                 b'10 Foolproof Tips for Better Sleep'
4     b"The 50 Coolest Jerseys You Didn t Know Exist...
5                           b'Genital Herpes Treatment'
6                   b'fashion lane American Wild Child'
7     b'Racing For Recovery by Dean Johnson racing f...
8                 b'Valet The Handbook 31 Days 31 days'
9         b'Cookies and Cream Brownies How Sweet It Is'
10    b'Business Financial News Breaking US Internat...
11    b'A Tip of the Cap to The Greatest Iron Man of...
12                     b'9 Foods That Trash Your Teeth'
13                                                  b''
14    b'French Onion Steaks with Red Wine Sauce fren...
15    b'Izabel Goulart Swimsuit by Kikidoll 2012 Spo...
16                b'Liquid Mountaineering The Awesomer'
17                                              

In [14]:
boilerPlateBodies.head(20)

0     b'A sign stands outside the International Busi...
1     b'And that can be carried on a plane without t...
2     b'Apples The most popular source of antioxidan...
3     b'There was a period in my life when I had a l...
4     b"Jersey sales is a curious business Whether y...
5     b'Genital herpes is caused by herpes simplex v...
6     b'Our favorite summer holiday is just around t...
7     b'Racing For Recovery is the growing idea that...
8     b"Resolutions are for suckers Instead of swear...
9     b"More brownies It seems that I can t get thro...
10    b'Faced with a stream of financial scandals th...
11    b"As you surely know by now Brett Favre s NFL ...
12    b'There are the 700 different types of germs i...
13                                                  b''
14    b'Really this is just an excuse to get drunk O...
15    b'TM 2012 Turner Broadcasting System Inc A Tim...
16    b"Link We re pretty skeptical about walking on...
17    b'The annual Chap Olympiad described as a 

In [16]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Vectorizer will by default remove punctuation and convert to lower case. We just need to strip the stop words before training our classifier.
vectorizer = TfidfVectorizer(min_df=2,stop_words='english',binary=False, analyzer='word', use_idf = True, smooth_idf = True, sublinear_tf = True) # use binary option to check for instances instead of summing instances
bodyVect = vectorizer.fit_transform(boilerPlateBodies)
titleVect = vectorizer.fit_transform(boilerPlateTitles)

<10566x45345 sparse matrix of type '<class 'numpy.float64'>'
	with 1461233 stored elements in Compressed Sparse Row format>

In [18]:
def countText(text, pattern, relative = True):
    import re
    result = len(re.findall(pattern, text))
    if relative == True:
        result = result / np.double(countWords(text))
    if text == '':
        result = 0
    return result

In [19]:
def countWords(text):
    import nltk
    if text == '':
        result = 0
    else: 
        result = len(nltk.word_tokenize(text))
    return result

In [20]:
keyTerms = ['\,',
            '\!',
            'butter',
            'water',
            'cup',
            'degree',
            'recipe'
            'comment',
            'email',
            'contact',
            '[0-9]{4}',
            '\|',
            '\d',
            '@',
            'calories',
            'sign?in',
            'log?in',
            'register'
            ]

In [22]:
boilerPlateBodies.apply?

In [21]:
textFeatures = pd.DataFrame(index = df.index)
for term in keyTerms:
    textFeatures['bpBodyContains_' + term] = boilerPlateBodies.apply(countText, pattern = term, relative = True)
    textFeatures['bpTitleContains_' + term] = boilerPlateTitles.apply(countText, pattern = term, relative = True)

TypeError: cannot use a string pattern on a bytes-like object

In [10]:
def getUrlDomain(url):
    import urlparse
    import re
    parsed = urlparse.urlparse(url)
    domain = re.sub('^www\.', '', parsed.netloc).split('.')
    result = domain[len(domain)-1]
    return result

def getName(url):
    import urlparse
    import re
    parsed = urlparse.urlparse(url)
    domain = re.sub('^www\.', '', parsed.netloc).split('.')
    result = domain[0]
    return result

def countPaths(url):
    import urlparse
    import re
    parsed = urlparse.urlparse(url)
    domain = re.findall('\/',parsed.path)
    return len(domain)

In [11]:
domains = df.url.apply(getUrlDomain)
webNames = df.url.apply(getName)
numPaths = df.url.apply(countPaths)

In [12]:
domains_df = pd.get_dummies(domains, prefix = 'DUM')
webNames_df = pd.get_dummies(webNames, prefix = 'DUM')
alchemyCategories_df = pd.get_dummies(df.alchemy_category, prefix = 'DUM')
numPaths_df = pd.get_dummies(numPaths, prefix = 'DUM')

In [13]:
originalFeatureList =['avglinksize','commonlinkratio_1','commonlinkratio_2','commonlinkratio_3','commonlinkratio_4', 
           'compression_ratio','embed_ratio','frameTagRatio', 'framebased', 'hasDomainLink', 'html_ratio', 'image_ratio', 
           'is_news', 'lengthyLinkDomain', 'linkwordscore', 'news_front_page', 'non_markup_alphanum_characters', 'numberOfLinks',
           'numwords_in_url', 'parametrizedLinkRatio', 'spelling_errors_ratio']

originalFeatures = df[originalFeatureList]

In [14]:
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [15]:
from sklearn.decomposition import TruncatedSVD

titleDecomposed = TruncatedSVD(random_state = 123, n_components = 10).fit_transform(titleVect)
bodyDecomposed = TruncatedSVD(random_state = 123, n_components = 50).fit_transform(bodyVect)

textDecomposed = np.hstack([titleDecomposed,bodyDecomposed])
textDecomposed_training = textDecomposed[df.setType == 'training']
textDecomposed_test = textDecomposed[df.setType == 'test']


# used for estimating alch cats
titleDecomposed2 = TruncatedSVD(random_state = 123, n_components = 100).fit_transform(titleVect)
bodyDecomposed2 = TruncatedSVD(random_state = 123, n_components = 500).fit_transform(bodyVect)
textDecomposed2 = np.hstack([titleDecomposed2,bodyDecomposed2])



In [16]:
# Construct Training and Test Data
originalFeatures_training = originalFeatures[df.setType == 'training']
textFeatures_training = textFeatures[df.setType == 'training']
domains_training = domains_df[df.setType == 'training']
webNames_training = webNames_df[df.setType == 'training']
numPaths_training = numPaths_df[df.setType == 'training']
alchemyCategories_training = alchemyCategories_df[df.setType == 'training']

Y = df[df.setType == 'training'].label
X = np.concatenate((originalFeatures_training.values, textFeatures_training, numPaths_training,
                    domains_training.values, webNames_training.values,alchemyCategories_training.values), 1)
X[X == '?'] = 0


# Construct ALTERNATE Training and Test Data
X_alt = np.hstack([X, textDecomposed_training])

In [17]:
originalFeatures_test = originalFeatures[df.setType == 'test']
textFeatures_test = textFeatures[df.setType == 'test']
domains_test = domains_df[df.setType == 'test']
webNames_test = webNames_df[df.setType == 'test']
numPaths_test = numPaths_df[df.setType == 'test']
alchemyCategories_test = alchemyCategories_df[df.setType == 'test']

X_Test = np.concatenate((originalFeatures_test.values, textFeatures_test, numPaths_test,
                    domains_test.values, webNames_test.values,alchemyCategories_test.values), 1)
X_Test[X_Test == '?'] = 0

X_Test_alt = np.hstack([X_Test, textDecomposed_test])

In [18]:
submission = pd.DataFrame(df[df.setType == 'test'].urlid)

In [164]:
def crossVal(clf, X_data, Y_data, NFOLDS = 10): 
    import sklearn.ensemble as ens
    from sklearn import metrics
    from sklearn.cross_validation import StratifiedKFold

    skf = StratifiedKFold(Y_data, n_folds = NFOLDS)
    auc = []
    for train_index, test_index in skf:
        x_train, x_test = X_data[train_index], X_data[test_index]
        y_train, y_test = Y_data[train_index], Y_data[test_index]
        
        clf.fit(x_train, y_train)
        
        try: 
            probs = clf.predict_proba(x_test)
            y_hat = probs[:,1]
        except:
            y_hat = clf.predict(x_test)
        auc.append(metrics.roc_auc_score(y_test, y_hat))
    print auc
    print np.mean(auc)
    return auc

In [237]:
elasticCV = ElasticNetCV(l1_ratio= [.1, .5, .7, .9, .95, .99, 1], n_jobs = 2)
elasticCV.fit(X.astype(np.float),Y)
alpha_star = elasticCV.alpha_
l1_ratio_star = elasticCV.l1_ratio_

ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
       l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000,
       n_alphas=100, n_jobs=1, normalize=False, precompute='auto',
       tol=0.0001, verbose=0)

In [239]:
# l1: 0.8, alpha = 1

clf_EN_1 = ElasticNet(alpha = alpha_star, l1_ratio=l1_ratio_star)
results = crossVal(clf_EN_1,X.astype(np.float),Y,10)
print 'alpha: ' + str(alpha_star)
print 'rho: ' + str(l1_ratio_star)
print results

alpha: 0.589482653704
rho: 0.1
[0.62111842105263071, 0.6539254385964911, 0.62657163742690059, 0.64047514619882906, 0.59868421052631571, 0.62637056581647621, 0.62775579009088234, 0.65041776605101154, 0.57822486074464952, 0.6257147045887701]


In [24]:
from sklearn.linear_model import LogisticRegression

In [22]:
rf = RandomForestClassifier(criterion = 'gini',random_state = 123,
                                 min_samples_leaf = 2, min_samples_split = 6, n_estimators = 300)

In [25]:
lr =  LogisticRegression(random_state = 123, penalty = 'l1', C = 1)

In [22]:
rf.fit(X_alt, Y)
lr.fit(X_alt, Y)

NameError: name 'alt' is not defined

In [24]:
probs_rf = rf.predict_proba(X_Test_alt)[:,1]
probs_lr = lr.predict_proba(X_Test_alt.astype(np.float))[:,1]
finalPrediction = np.vstack([probs_rf, probs_lr]).mean(0)
submission['label'] = finalPrediction.T
submission.to_csv('submission_21.csv', index = False, sep = ',')

### Train chosen classifier and predict on test set

In [159]:
finalPrediction = np.vstack([y_hat_GBM, y_hat_RF]).mean(0)

In [159]:
print RF_Best_alt_cv
print np.mean(RF_Best_alt_cv)

[0.86701754385964902, 0.85896929824561297, 0.88310672514619903, 0.87345760233918102, 0.86160818713450316, 0.89448841981823568, 0.86003371445323962, 0.87238346525945376, 0.8564057461155089, 0.85340126081219814]
0.868087196318


In [ ]:
parameters3 = {'learning_rate': [0.05, 0.1, 0.25, 0.5, 0.75],
              'n_estimators' : [100, 300, 500, 700, 1000, 2000, 4000],
              'max_depth' : [2,3,4,5,10,15,20,50,100],
              'min_samples_split' : [2,4,6,8,10],
              'min_samples_leaf' : [1,2,5],
              'subsample' : [0.5, 0.75, 1]}
clfGBMGrid = GridSearchCV(GradientBoostingClassifier(random_state = 123), parameters3).fit(X,Y)

In [27]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(random_state = 123, penalty = 'l1', C = 1)

In [56]:
results = crossVal(lr, X_alt.astype(np.float), Y, 10)
print results
print np.mean(results)

[0.86211257309941691, 0.8669809941520461, 0.88327485380117032, 0.88035087719298522, 0.85097953216374278, 0.88353855174435636, 0.86282615068894819, 0.877814423922603, 0.84919378481383645, 0.85499193666617901]
0.867206367825


In [26]:
from sklearn.ensemble import AdaBoostClassifier

In [27]:
ada_rf = AdaBoostClassifier(base_estimator=rf)

In [29]:
results = crossVal(ada_rf, X_alt, Y)
print results
print np.mean(results)

[0.87054824561403465, 0.8634429824561427, 0.88392543859649209, 0.87408625730994072, 0.86663742690058687, 0.89635004397537366, 0.86713573732043414, 0.87586484901788386, 0.85677953679273, 0.85437619117431451]
0.870914670916


In [19]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.metrics import confusion_matrix

In [20]:
alch_cat_training = df[(df.alchemy_category != '?') & (df.alchemy_category != 'unknown') & (df.setType == 'training')].alchemy_category

In [21]:
predictors = textDecomposed2[(df.alchemy_category != '?') & (df.alchemy_category != 'unknown') & (df.setType == 'training')]

In [22]:
alch_cats = alch_cat_training.unique()

In [23]:
i = 0
for item in alch_cats:
    alch_cat_training[alch_cat_training == item] = i
    i += 1

In [24]:
clf2 = BernoulliNB(alpha = 0.5, fit_prior=True)
clf2.fit(predictors, alch_cat_training.astype(np.int))
predicted = clf2.predict(predictors)
result = pd.DataFrame(confusion_matrix(alch_cat_training.astype(np.int), predicted), columns = alch_cats, index = alch_cats)
print result
estAlchCat = clf2.predict(textDecomposed2)

                    business  recreation  health  sports  arts_entertainment  \
business                 482         159      37      10                  83   
recreation               209         680      37      36                 118   
health                    26          18     385       6                  19   
sports                    12          15      13     280                  28   
arts_entertainment       141         121      27      62                 473   
science_technology        47          27      28       3                  24   
gaming                     8          10       6      12                  12   
culture_politics          36          60      17       8                  43   
computer_internet         14          39       6       3                  28   
law_crime                  5           3       0       0                   7   
religion                   9           8       8       3                  16   
weather                    0           0

In [25]:
estAlchCat[(df.alchemy_category != '?') & (df.alchemy_category != 'unknown') & (df.setType == 'training')] = alch_cat_training[(df.alchemy_category != '?') & (df.alchemy_category != 'unknown') & (df.setType == 'training')]

In [26]:
alchemyCategories2_df = pd.get_dummies(estAlchCat, prefix = 'DUM')
alchemyCategories2_training = alchemyCategories2_df[df.setType == 'training']
alchemyCategories2_test = alchemyCategories2_df[df.setType == 'test']

In [27]:
X_alt_2 = X = np.hstack([originalFeatures_training.values, textFeatures_training, numPaths_training,
                    domains_training.values, webNames_training.values,alchemyCategories2_training.values, textDecomposed_training])

X_Test_2 = np.hstack([originalFeatures_test.values, textFeatures_test, numPaths_test,
                    domains_test.values, webNames_test.values,alchemyCategories2_test.values, textDecomposed_test])

X_alt_2[X_alt_2 == '?'] = 0
X_Test_2[X_Test_2 == '?'] = 0

In [154]:
results = crossVal(rf, X_alt_2, Y)
print results
print np.mean(results)

[0.86582602339181425, 0.86538011695906492, 0.88462719298245562, 0.87537280701754439, 0.86476608187134452, 0.89866608032834916, 0.86748021108179463, 0.87712547639988148, 0.85537965406039185, 0.85117284855592923]
0.870579649265


In [153]:
results = crossVal(lr, X_alt_2.astype(np.float), Y)
print results
print np.mean(results)

[0.86722953216374221, 0.86760964912280591, 0.88595760233918197, 0.87606725146198894, 0.85593567251462133, 0.88792143066549356, 0.86999413661682912, 0.87944151275285798, 0.85029316915860287, 0.85405365782143294]
0.869450361462


In [28]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbm = GradientBoostingClassifier(random_state=123)

In [167]:
results = crossVal(gbm, X_alt_2, Y)

[0.86604532163742776, 0.87507309941520584, 0.88288377192982503, 0.8753106725146188, 0.87012792397660765, 0.89775359132219124, 0.87153693931398435, 0.87915200820873529, 0.85851289944297993, 0.85981527635244115]
0.873621150411


In [177]:
crossVal(lr, X_alt_2.astype(np.float), Y)

[0.86722953216374221, 0.86760964912280591, 0.88595760233918197, 0.87606725146198894, 0.85593567251462133, 0.88792143066549356, 0.86999413661682912, 0.87944151275285798, 0.85029316915860287, 0.85405365782143294]
0.869450361462


[0.86722953216374221,
 0.86760964912280591,
 0.88595760233918197,
 0.87606725146198894,
 0.85593567251462133,
 0.88792143066549356,
 0.86999413661682912,
 0.87944151275285798,
 0.85029316915860287,
 0.85405365782143294]

In [ ]:
gbm_parameters = {'n_estimators':[20, 50,100,200,500,1000]}
gbm = GradientBoostingClassifier(random_state = 123, max_depth = 6)
gbm_grid = GridSearchCV(gbm, gbm_parameters, verbose = 1, n_jobs = 4).fit(X_alt_2,Y)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


In [ ]:
gbm_grid.best_estimator_